# Creación de un Data Set desde webscrapping

A continuación, se explican los pasos realizados para armar un conjunto de datos compuestos por textos de 3 temas de interés, que luego se utilizarán para el entrenamiento de un modelo de clasificación de textos.


## Preparación del ambiente de trabajo

Clonar el proyecto desde GitHub haciendo `git clone ` en la consola, ubicando previamente la carpeta donde se quiere trabajar.

Para preparación del entorno de trabajo en Visual Studio Code se deben hacer los siguients pasos:

* Crear entorno Python `-m venv env`
* Ingresar al entorno haciendo `venv\Scripts\activate`
* Instalar dependencias con `pip install -r requirements.txt`

## Importaciones y descargas necesarias

In [10]:
# Importaciones a usar

#import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
# Descargas necesarias
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Creación del conjunto de datos

### Creación del listado de URL para armar el conjunto de datos

En primer lugar se necesita crear el archivo `DataSet.csv` que se puede crear con ChatGPT. Por ejemplo, para este proyecto se utilizó el siguiente prompt para solicitar un listado de URL por temas de interés:

"*Sos un machine learning developer, y estas haciendo un data set para una clasificación de texto. Necesitas armar el conjunto de datos. Necesito que me des 50 links de wikipedia en ingles, relacionados con el tema Meteorología.
Me los tenes que dar en formato csv con dos columnas, la primera con las URL y la segunda con la Clase, que en este caso sería Meteorology.*"

Solicitarle continuar la respuesta hasta que finalice de escribir las URL, si fuera necesario. Cuando finalice, se le puede decir:

"*Dame lo mismo para biologia*"

De esta manera, se puede solicitar todas las categorías de interés. Para este proyecto se usaron: Meteorología, Biología y Deporte.

Guardar todos los links que proporciona ChatGPT en un solo archivo .csv con la precaución de nombrar las columnas como URL y Clase.

### Creación del DataFrame con los textos extraídos de las URL

Con la función `obtener_registros_por_parrafo` se recorre el archivo `DataSet.csv`, obtiene los textos ubicados entre etiquetas `<p>` y los guarda en un DataFrame llamado `registros_df`.



In [6]:
def obtener_registros_por_parrafo(archivo):
    '''
    Permite abrir un archivo .csv que contiene una columna con las URL desde donde se quiere extrae textos y
    otra columna con las Clase (categoría) a la que pertenece ese texto.
    La función recorre cada URL y toma el texto que se encuentra en las etiquetas <p>.
    Finalmente lo guarda en un DataFrame de pandas.
    '''
    # Lee el archivo de CSV con pandas
    df = pd.read_csv(archivo, encoding='latin1')

    # Lista vacía para guardar los registros
    registros = []

    # Itera sobre cada registro del df
    for _, row in df.iterrows():
        url = row["URL"]
        clase = row["Clase"]

        # Toma los parrafos si es que la URL funciona
        req = requests.get(url)
        if req.status_code == 200:
            html = BeautifulSoup(req.text, "html.parser")
            parrafos = [para.text for para in html.find_all("p")]

            for parrafo in parrafos:
                # Divide el párrafo en sentencias usando los saltos de línea
                sentencias = parrafo.split('\n')

                for sentencia in sentencias:
                    # Elimina espacios en blanco al inicio y final de la sentencia
                    sentencia = sentencia.strip()

                    # Agrega un registro por cada sentencia conservando la URL y la clase
                    registros.append({"URL": url, "Clase": clase, "Sentencia": sentencia})

    # Crea el DtaFrame final con los registros obtenidos
    registros_df = pd.DataFrame(registros)
    return registros_df


In [7]:
# Ejecutamos la función y visualizamos el DataFrame
archivo = "./DataSet.csv"
df_registros = obtener_registros_por_parrafo(archivo)
df_registros

,URL,Clase,Sentencia
0,https://en.wikipedia.org/wiki/Meteorology,Meteorology,
1,https://en.wikipedia.org/wiki/Meteorology,Meteorology,
2,https://en.wikipedia.org/wiki/Meteorology,Meteorology,Atmospheric physics
3,https://en.wikipedia.org/wiki/Meteorology,Meteorology,Atmospheric dynamics (category)
4,https://en.wikipedia.org/wiki/Meteorology,Meteorology,
...,...,...,...
19265,https://en.wikipedia.org/wiki/Sports_equipment,Sport,
19266,https://en.wikipedia.org/wiki/Sports_equipment,Sport,Vehicles (sometimes specialized) are used as e...
19267,https://en.wikipedia.org/wiki/Sports_equipment,Sport,
19268,https://en.wikipedia.org/wiki/Sports_equipment,Sport,Small vehicles with flatbeds are often used to...


## Limpieza de los textos

La función `limpiar_df` realiza la limpieza de los textos extraídos de las URL y guardados en el DataFrame del punto anterior.

La limpieza consiste en:
* Convertir todos los textos a minúscula
* Eliminar stopswords, números y signos de puntuación
* Obtener los lemas de las palabras
* Crea sentencias con los saltos de línea, para obtener textos mas cortos.
* Eliminar registros con menos de 10 palabras

Retorna el DataFrame limpio donde podemos averiguar la cantidad de registros, la longitud máxima de las sentencias y la longitud promedio. Estos últimos valores los necesitamos conocer para saber dónde podemos truncar las sentencias.

In [11]:
def limpiar_df(df):
    '''
    Permite limpiar un DataFrame, convirtiendo todo a minúsculas, eliminando stopswords, números y signos de puntuación,
    obteniendo los lemas de las palabras y eliminando registros cortos de menos de 10 palabras.
    Retorna un DataFrame limpio.
    '''
    # Convertir todo a minúsculas
    df['Sentencia'] = df['Sentencia'].str.lower()

    # Eliminar stopwords, números y signos de puntuación
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    df['Sentencia'] = df['Sentencia'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words and not word.isnumeric()]))

    # Genera sentencias por cada salto de línea
    df['Sentencia'] = df['Sentencia'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Eliminar registros con menos de 10 palabras
    df['Num_Words'] = df['Sentencia'].apply(lambda x: len(re.findall(r'\w+', x)))
    df = df[df['Num_Words'] >= 10].drop(columns='Num_Words')

    return df


In [12]:
# Generamos el DataFrame limpio y lo vemos
df_limpio = limpiar_df(df_registros)
df_limpio

,URL,Clase,Sentencia
10,https://en.wikipedia.org/wiki/Meteorology,Meteorology,meteorology branch atmospheric science which i...
12,https://en.wikipedia.org/wiki/Meteorology,Meteorology,meteorological phenomenon observable weather e...
14,https://en.wikipedia.org/wiki/Meteorology,Meteorology,meteorology climatology atmospheric physics at...
16,https://en.wikipedia.org/wiki/Meteorology,Meteorology,word meteorology ancient greek μετέωρος metéōr...
18,https://en.wikipedia.org/wiki/Meteorology,Meteorology,early attempt predicting weather often related...
...,...,...,...
19260,https://en.wikipedia.org/wiki/Sports_equipment,Sport,protective equipment often worn sport includin...
19262,https://en.wikipedia.org/wiki/Sports_equipment,Sport,example training equipment include swiss balls...
19264,https://en.wikipedia.org/wiki/Sports_equipment,Sport,special sport equipment equipment usually worn...
19266,https://en.wikipedia.org/wiki/Sports_equipment,Sport,vehicle sometimes specialized used equipment s...
